In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
from nltk.parse.generate import generate
from nltk import CFG
from nltk.grammar import Nonterminal
from tqdm import tqdm

from eig.battleship import Parser

from battleship.grammar import BattleshipGrammar
from battleship.scoring import compute_score_parallel
from battleship.board import Board

# Battleship grammar

In [ ]:
grammar = BattleshipGrammar(include_lambdas=False)

## Enumeration

In [ ]:
valid, invalid = grammar.generate(n=int(1e6), depth=5, start=Nonterminal('A'))
print('Valid programs:', len(valid))
print('Invalid programs:', len(invalid))

In [ ]:
# Pass `enforce_type=False` to `Parser.parse` to allow all toplevel types
program = Parser.parse("( and TRUE ( touch Red Blue ) )", enforce_type=False)

In [ ]:
program.to_dict()

## Sampling

In [ ]:
n = int(1e6)
samples = [grammar.sample() for _ in tqdm(range(n))]
# Remove None samples
samples = list(set(s for s in samples if s is not None))
print('Samples:', len(samples))

## Scoring

In [ ]:
TRIAL_IDS = range(3, 19)

def evaluate_samples_all_trials(samples):
    # Deduplicate samples
    samples = list(set([x for x in samples if x is not None]))

    # Create dataframe to store scores
    df_scores = pd.DataFrame({"program": samples})
    df_scores["length"] = df_scores["program"].apply(lambda p: len(p))

    for trial_id in TRIAL_IDS:
        print(f"Evaluating trial {trial_id}")
        board = Board.from_trial_id(trial_id)
        scores = compute_score_parallel(programs=samples, board=board, processes=os.cpu_count() // 6)
        df_scores[f"score_{trial_id}"] = scores

    return df_scores

df_scores = evaluate_samples_all_trials(samples[:10000])

In [ ]:
board = Board.from_trial_id(4)
compute_score_parallel(programs=samples, board=board, processes=os.cpu_count() // 4)

In [ ]:
df_scores